In [1]:
! pip install --quiet datasets
! pip install --quiet tabulate


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from tabulate import tabulate
from collections import Counter
from datasets import load_dataset

# from huggingface_hub import notebook_login
# notebook_login()

# Set Pandas display options for better DataFrame output
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', 1000)        # Set the width to a larger value
pd.set_option('display.max_colwidth', 100)  # Set max column width to display wider content


/Users/jericnarte/.pyenv/versions/thesis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_path = './dataset/rice_leaf_disease_split'
hub_path = 'cvmil/rice-leaf-disease-augmented-v2'

In [4]:
# Load the dataset
dataset = load_dataset('imagefolder', data_dir=dataset_path)
print(f"Dataset: {dataset}")

Generating train split: 8000 examples [00:00, 30941.09 examples/s]
Generating validation split: 336 examples [00:00, 27496.12 examples/s]
Generating test split: 332 examples [00:00, 22008.99 examples/s]


Dataset: DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 336
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 332
    })
})


In [5]:
# Assuming label mapping and label names are available
label_mapping = dataset['train'].features['label'].int2str
labels = dataset['train'].features['label'].names

# Count the number of samples per class in each split
train_counts = Counter(dataset['train']['label'])
validation_counts = Counter(dataset['validation']['label'])
test_counts = Counter(dataset['test']['label'])

# Create a DataFrame for the counts, ordered by label name
data = {
    'ID': list(range(len(labels))),
    'Label': labels,
    'Training': [train_counts[i] if i in train_counts else 0 for i in range(len(labels))],
    'Validation': [validation_counts[i] if i in validation_counts else 0 for i in range(len(labels))],
    'Test': [test_counts[i] if i in test_counts else 0 for i in range(len(labels))]
}
df = pd.DataFrame(data)

# Print the DataFrame with borders using tabulate
print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

+------+------------------------+------------+--------------+--------+
|   ID | Label                  |   Training |   Validation |   Test |
+======+========================+============+==============+========+
|    0 | Bacterial Leaf Blight  |       1000 |           36 |     36 |
+------+------------------------+------------+--------------+--------+
|    1 | Brown Spot             |       1000 |           53 |     52 |
+------+------------------------+------------+--------------+--------+
|    2 | Healthy Rice Leaf      |       1000 |           32 |     31 |
+------+------------------------+------------+--------------+--------+
|    3 | Leaf Blast             |       1000 |           60 |     60 |
+------+------------------------+------------+--------------+--------+
|    4 | Leaf scald             |       1000 |           36 |     36 |
+------+------------------------+------------+--------------+--------+
|    5 | Narrow Brown Leaf Spot |       1000 |           23 |     22 |
+-----

In [6]:
# upload dataset to huggingface
dataset.push_to_hub(hub_path, private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:07<00:00,  7.96s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/cvmil/rice-leaf-disease-augmented-v2/commit/2da8d5ff8101d8cc4792f92b4609652c50e5b1f1', commit_message='Upload dataset', commit_description='', oid='2da8d5ff8101d8cc4792f92b4609652c50e5b1f1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/cvmil/rice-leaf-disease-augmented-v2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='cvmil/rice-leaf-disease-augmented-v2'), pr_revision=None, pr_num=None)